<a href="https://colab.research.google.com/github/rautaditya2606/Rossman-Deployed/blob/main/Roassman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets --quiet
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import matplotlib
%matplotlib inline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
import opendatasets as od
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree, export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
from xgboost import XGBRegressor, plot_tree
from matplotlib.pylab import rcParams
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import joblib
from google.colab import files

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10,6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'
pd.set_option("display.max_columns", 120)
pd.set_option("display.max_rows", 120)

In [ ]:
od.download('https://www.kaggle.com/c/rossmann-store-sales')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

In [ ]:
ross_df = pd.read_csv('./rossmann-store-sales/train.csv', low_memory=False)
store_df = pd.read_csv('./rossmann-store-sales/store.csv')
test_df = pd.read_csv('./rossmann-store-sales/test.csv')
submission_df = pd.read_csv('./rossmann-store-sales/sample_submission.csv')

In [ ]:
merged_df = ross_df.merge(store_df, how='left', on='Store')
merged_test_df = test_df.merge(store_df, how='left', on='Store')

In [ ]:
merged_df = merged_df[merged_df['Open'] == 1].copy()
def split_date(df):
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['Year']= df.Date.dt.year
    df['Month'] = df.Date.dt.month
    df['Day'] = df.Date.dt.day
    df['WeekOfYear'] = df.Date.dt.isocalendar().week

In [ ]:
split_date(merged_df)
split_date(merged_test_df)

In [ ]:
merged_df

In [ ]:
  def comp_month(df):
      df['CompetitionOpen'] = 12 * (df.Year - df.CompetitionOpenSinceYear) + (df.Month - df.CompetitionOpenSinceMonth)
      df['CompetitionOpen'] = df['CompetitionOpen'].map(lambda x : 0 if x < 0 else x).fillna(0)

In [ ]:
comp_month(merged_df)
comp_month(merged_test_df)

In [ ]:
merged_df[['Date',  'CompetitionDistance', 'CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth', 'CompetitionOpen']]

In [ ]:
def check_promo_month(row):
    month2str = {
        1:'Jan',
        2:'Feb',
        3:'Mar',
        4:'Apr',
        5:'May',
        6:'Jun',
        7:'Jul',
        8:'Aug',
        9:'Sept',
        10:'Oct',
        11:'Nov',
        12:'Dec'
    }
    try:
        months = (row['PromoInterval'] or '').split(',')
        if row['Promo2Open'] and month2str[row['Month']] in months:
            return 1
        else:
            return 0
    except Exception:
        return 0


def promo_cols(df):
    df['Promo2Open'] = 12 * (df.Year - df.Promo2SinceYear) + (df.WeekOfYear - df.Promo2SinceWeek) * 7/30.5
    df['Promo2Open'] = df['Promo2Open'].map(lambda x : 0 if x < 0 else x).fillna(0) * df['Promo2']
    df['IsPromo2Month'] = df.apply(check_promo_month, axis=1) * df['Promo2']

In [ ]:
promo_cols(merged_df)
promo_cols(merged_test_df)

In [ ]:
merged_df[['Date', 'Promo2', 'Promo2SinceYear', 'Promo2SinceWeek', 'PromoInterval', 'Promo2Open', 'IsPromo2Month']]

In [ ]:
merged_df = merged_df[merged_df['Open'] == 1].copy()

In [ ]:
input_cols = ['Store', 'DayOfWeek', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpen', 'Day', 'Month', 'Year', 'WeekOfYear', 'Promo2', 'Promo2Open', 'IsPromo2Month']
target_col = 'Sales'

In [ ]:
inputs = merged_df[input_cols].copy()
target = merged_df[target_col].copy()
test_inputs = merged_test_df[input_cols].copy()

In [ ]:
numeric_cols = ['Store','Promo', 'SchoolHoliday', 'CompetitionDistance', 'CompetitionOpen', 'Promo2', 'Promo2Open', 'IsPromo2Month', 'Day','Month', 'Year', 'WeekOfYear']
categorical_cols = ['DayOfWeek', 'StateHoliday', 'StoreType', 'Assortment']

In [ ]:
inputs[numeric_cols].isna().sum()

In [ ]:
test_inputs[numeric_cols].isna().sum()

In [ ]:
max_dist = inputs['CompetitionDistance'].max()
max_dist

In [ ]:
inputs['CompetitionDistance'] = inputs['CompetitionDistance'].fillna(max_dist*2)
test_inputs['CompetitionDistance'] = test_inputs['CompetitionDistance'].fillna(max_dist*2)

In [ ]:
scaler = MinMaxScaler().fit(inputs[numeric_cols])

In [ ]:
inputs[numeric_cols] = scaler.transform(inputs[numeric_cols])
test_inputs[numeric_cols] = scaler.transform(test_inputs[numeric_cols])

In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(inputs[categorical_cols])
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))

In [ ]:
inputs[encoded_cols] = encoder.transform(inputs[categorical_cols])
test_inputs[encoded_cols] = encoder.transform(test_inputs[categorical_cols])

In [ ]:
x_train = inputs[numeric_cols+encoded_cols]
x_test = test_inputs[numeric_cols+encoded_cols]

In [ ]:
x_train

In [ ]:
# model = XGBRegressor(n_jobs=-1, random_state=42, n_estimators = 100, max_depth = 4).fit(x_train, target)

In [ ]:
# train_pred = model.predict(x_test)
# train_pred

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
# train_pred = model.predict(x_train)
# train_rmse = rmse(target, train_pred)
# print("Train RMSE:", train_rmse)

In [ ]:
# plt.hist(merged_df.Sales, bins=100)
# plt.show()

In [ ]:
# rcParams['figure.figsize'] = 30,30

In [ ]:
# plot_tree(model, rankdir='LR', num_trees=0)

In [ ]:
# plot_tree(model, rankdir='LR', num_trees=1)

In [ ]:
# plot_tree(model, rankdir='LR', num_trees=19)

In [ ]:
important_df =pd.DataFrame(
     {
    'Feature'  : x_test.columns,
    'Importance' : model.feature_importances_
}).sort_values('Importance', ascending=False)

In [ ]:
# important_df

In [ ]:
# plt.figure(figsize=(10,5))
# sns.barplot(data=important_df.head(10), x='Importance', y='Feature')

In [ ]:
# def train_and_evaluate(x_train, train_target, x_val, val_target, **params):
#   model = XGBRegressor(n_jobs=-1, random_state=42, **params).fit(x_train, train_target)
#   train_rmse = rmse(model.predict(x_train), train_target)
#   val_rmse = rmse(model.predict(x_val),val_target)
#   return model, train_rmse, val_rmse

In [ ]:
kfold= KFold(n_splits=5)

In [ ]:
# models = []
# for train_idxs, val_idxs in kfold.split(x_train):
#     x_tr, train_target = x_train.iloc[train_idxs], target.iloc[train_idxs]
#     x_val, val_target = x_train.iloc[val_idxs], target.iloc[val_idxs]

#     model, train_rmse, val_rmse = train_and_evaluate(
#         x_tr, train_target, x_val, val_target,
#         n_estimators=200, max_depth=10
#     )
#     models.append(model)
#     print('Train RMSE: {}, Validation RMSE: {}'.format(train_rmse, val_rmse))


In [ ]:
model = XGBRegressor(n_jobs = -1, n_estimators = 200, max_depth = 7).fit(x_train, target)

In [ ]:
train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
train_pred, test_pred

In [ ]:
submission_df['Sales'] = test_pred

In [ ]:
submission_df['Sales'] = model.predict(x_test)
submission_df['Sales'] = np.nan_to_num(submission_df['Sales'], nan=0)

In [ ]:
test_df['Open'] = test_df['Open'].fillna(1)
submission_df['Sales'] = submission_df['Sales'] * test_df['Open']

In [ ]:
submission_df['Sales'].isna().sum()

In [ ]:
submission_df.shape
test_df.shape
submission_df.index.equals(test_df.index)

In [ ]:
submission_df.to_csv('submission.csv', index=None)

In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink('submission.csv')

In [ ]:
# plt.figure(figsize=(10,5))
# sns.histplot(merged_df['Sales'], bins=50, kde=True)
# plt.title("Sales Distribution Across Stores")
# plt.xlabel("Sales")
# plt.ylabel("Count")
# plt.tight_layout()
# plt.show()

In [ ]:
store_static_dict = store_df.set_index('Store').to_dict(orient='index')

In [ ]:
user_input = {
    "Store": 85,
    "Date": "2015-06-15",
    "Promo": 1,
    "StateHoliday": "0",
    "SchoolHoliday": 1
}

In [ ]:
store_metadata = store_static_dict[user_input['Store']]

In [ ]:
month2str = {
    1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',
    7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'
}

In [ ]:
def decode_input(user_input, store_static_dict):
    store_id = user_input['Store']
    store_metadata = store_static_dict[store_id]

    # optimise date
    date = pd.to_datetime(user_input['Date'])
    year = date.year
    month = date.month
    day = date.day
    week_of_year = date.isocalendar().week
    day_of_week = date.dayofweek + 1


     # handle missing competition data
    comp_year = store_metadata['CompetitionOpenSinceYear'] if pd.notnull(store_metadata['CompetitionOpenSinceYear']) else year
    comp_month = store_metadata['CompetitionOpenSinceMonth'] if pd.notnull(store_metadata['CompetitionOpenSinceMonth']) else month

    competition_open = max(0, 12 * (year - comp_year) + (month - comp_month))

    # handle Promo2
    if store_metadata['Promo2'] == 1 and pd.notnull(store_metadata['Promo2SinceYear']) and pd.notnull(store_metadata['Promo2SinceWeek']):
        promo2_open = 12 * (year - store_metadata['Promo2SinceYear']) + ((week_of_year - store_metadata['Promo2SinceWeek']) * 7 / 30.5)
        promo2_open = max(0, promo2_open)
    else:
        promo2_open = 0

    if store_metadata['PromoInterval'] and store_metadata['Promo2'] == 1:
        promo_months = store_metadata['PromoInterval'].split(',')
        is_promo2_month = int(month2str[month] in promo_months)
    else:
        is_promo2_month = 0

    #final input
    decoded_input = {
        'Store': store_id,
        'DayOfWeek' : day_of_week,
        'Promo': user_input['Promo'],
        'StateHoliday' : user_input['StateHoliday'],
        'SchoolHoliday' : user_input['SchoolHoliday'],
        'StoreType' : store_metadata['StoreType'],
        'Assortment': store_metadata['Assortment'],
        'CompetitionDistance': store_metadata['CompetitionDistance'],
        'CompetitionOpen': competition_open,
        'Day':day,
        'Month':month,
        'Year': year,
        'WeekOfYear': week_of_year,
        'Promo2' : store_metadata['Promo2'],
        'Promo2Open': promo2_open,
        'IsPromo2Month': is_promo2_month
        }

    decoded_input = pd.DataFrame([decoded_input])
    return decoded_input

In [ ]:
decoded_df = decode_input(user_input, store_static_dict)

In [ ]:
decoded_df

In [ ]:
decoded_df[numeric_cols] = scaler.transform(decoded_df[numeric_cols])
decoded_df[encoded_cols] = encoder.transform(decoded_df[categorical_cols])

In [ ]:
x_decoded = decoded_df[numeric_cols+encoded_cols]

In [ ]:
  decoded_pred = model.predict(x_decoded)
  decoded_pred

In [ ]:
joblib.dump(model, 'xgb_pipeline.joblib')
joblib.dump(scaler, 'scaler.joblib')
joblib.dump(encoder, 'encoder.joblib')
joblib.dump(store_static_dict, 'store_static_dict.joblib')

In [ ]:

files.download('xgb_pipeline.joblib')
files.download('scaler.joblib')
files.download('encoder.joblib')
files.download('store_static_dict.joblib')